# Estimación de dirección de arribo de fuentes sonoras 

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import gspread # Extension para leer el excel desde Drive
from oauth2client.service_account import ServiceAccountCredentials # Extensión para autentificación con Drive
from tdoa_py import estimation, simulation_room

### Nombre de la simulación

In [2]:
#Nombre de la simulación
simulation_name = "big_room_sweep_classic"

### Configuración de Gspreadsheet

En caso de no querer utilizar GSpreadsheet se puede armar un dataframe que contenga las siguientes columnas: 

sim_name	method	audio	room_x	room_y	room_z	rt60	snr_db	n_mics	mic_d	mic_z	mic_directivity	src_dist	src_z	src_az_deg	avg_angle_deg	theta_1	theta_2	theta_3	t_12	t_13	t_14	avg_tdoa	room_dim

In [4]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo 
# Pegar link del GSpreadsheet
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=1166684204#gid=1166684204")
sheet = spreadsheet.worksheet(simulation_name)

# Leer datos
data = sheet.get_all_records()
df_simulations = pd.DataFrame(data)

df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla luego
df_simulations.head() # Verificar el resultado

,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,src_az_deg,avg_angle_deg,theta_1,theta_2,theta_3,t_1,t_2,t_3,avg_tdoa,room_dim
0,big_room_sweep_0,classic,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,0,,,,,,,,,"(100, 100, 100)"
1,big_room_sweep_5,classic,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,5,,,,,,,,,"(100, 100, 100)"
2,big_room_sweep_10,classic,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,10,,,,,,,,,"(100, 100, 100)"
3,big_room_sweep_15,classic,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,15,,,,,,,,,"(100, 100, 100)"
4,big_room_sweep_20,classic,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,20,,,,,,,,,"(100, 100, 100)"


## Bloque de código para ejecutar todo directamente

Permite cargar los datos del Excel y procesa todo en tiempo real. Acá se puede elegir si guardar o no los audios con **save_audio = True**, en caso de ser falso se trabajará todo con numpy array con los datos de las señales procesadas.

In [5]:
# Iterar sobre el DataFrame 
for idx, sim in df_simulations.iterrows():
    src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
        wav_path=f'audios/anechoic/{sim["audio"]}',
        out_dir=f'audios/output/{simulation_name}/{sim["sim_name"]}',
        sim_name=sim["sim_name"],
        fs=48000,
        room_dim=sim["room_dim"],
        rt60=sim["rt60"],
        snr_db=sim["snr_db"],
        n_mics=sim["n_mics"],
        mic_d=sim["mic_d"],
        mic_z=sim["mic_z"],
        mic_directivity=sim["mic_directivity"],
        src_dist=sim["src_dist"],
        src_az_deg=sim["src_az_deg"],
        src_z=sim["src_z"],
        save_audio=True
    )
    
    avg_angle, avg_tdoa, angles_per_mic_ref, tdoas_per_mic_ref = estimation.estimate_doa(
        signals=mic_signals,
        fs=48000,
        d=sim["mic_d"],
        method=sim["method"]
    )

    # Guardar resultados en cada fila individual

    df_simulations.at[idx, 'avg_angle_deg'] = avg_angle
    df_simulations.at[idx, 'avg_tdoa'] = avg_tdoa
    df_simulations.at[idx, 'theta_1'] = angles_per_mic_ref[0]
    df_simulations.at[idx, 'theta_2'] = angles_per_mic_ref[1]
    df_simulations.at[idx, 'theta_3'] = angles_per_mic_ref[2]
    df_simulations.at[idx, 't_1'] = tdoas_per_mic_ref[0]
    df_simulations.at[idx, 't_2'] = tdoas_per_mic_ref[1]
    df_simulations.at[idx, 't_3'] = tdoas_per_mic_ref[2]

# # Convertir el DataFrame a una lista de listas para Google Sheets
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()

# Escribir en la hoja de cálculo
sheet.update(data)
print("DataFrame guardado correctamente en Google Sheets.")
df_simulations.head()



DataFrame guardado correctamente en Google Sheets.


,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,src_az_deg,avg_angle_deg,theta_1,theta_2,theta_3,t_1,t_2,t_3,avg_tdoa,room_dim
0,big_room_sweep_0,classic,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,0,0.0,0.0,0.0,0.0,0.000583,0.000438,0.000292,0.000438,"100,100,100"
1,big_room_sweep_5,classic,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,5,0.0,0.0,0.0,0.0,0.000583,0.000438,0.000292,0.000438,"100,100,100"
2,big_room_sweep_10,classic,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,10,1.380001,4.140004,0.0,0.0,0.000576,0.000438,0.000292,0.000435,"100,100,100"
3,big_room_sweep_15,classic,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,15,5.622148,9.23058,7.635865,0.0,0.000569,0.000427,0.000292,0.000429,"100,100,100"
4,big_room_sweep_20,classic,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,20,21.726984,21.726984,21.726984,21.726984,0.000542,0.000406,0.000271,0.000406,"100,100,100"


# Bloque de código para trabajar con los audios ya creados

In [ ]:
# Nombre de la simulación
folder_wav = "room_big_sweep"
simulation_name = "room_big_sweep_roth"

# Configurar credenciales para acceder a Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=0#gid=0")
sheet = spreadsheet.worksheet(simulation_name)

# Leer datos de la hoja de cálculo
df_simulations = pd.DataFrame(sheet.get_all_records())
df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla luego

# directorio base donde se guardan las simulaciones
base_audio_dir = f"audios/output/{folder_wav}"

# Iterar sobre el DataFrame y procesar cada simulación
for idx, sim in df_simulations.iterrows():
    sim_path = os.path.join(base_audio_dir, sim["sim_name"])  # Directorio de la simulación

    # Obtener todos los archivos de micrófonos en la simulación
    mic_files = sorted(glob.glob(os.path.join(sim_path, "*.wav")))
    print(mic_files)
    # Opción 1: Pasar rutas de archivos directamente
    avg_angle, avg_tdoa, angles, tdoas = estimation.estimate_doa(
        signals=mic_files,  # Pasamos las rutas de archivos directamente
        fs=48000,
        d=sim["mic_d"],
        method=sim["method"]
    )

    # Guardar resultados en el DataFrame
    df_simulations.at[idx, 'avg_angle_deg'] = avg_angle
    df_simulations.at[idx, 'avg_tdoa'] = avg_tdoa
    df_simulations.at[idx, 'theta_1'] = angles[0]
    df_simulations.at[idx, 'theta_2'] = angles[1]
    df_simulations.at[idx, 'theta_3'] = angles[2]
    df_simulations.at[idx, 't_12'] = tdoas[0]
    df_simulations.at[idx, 't_13'] = tdoas[1]
    df_simulations.at[idx, 't_14'] = tdoas[2]

# Convertir el DataFrame a formato lista para Google Sheets
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()

# Escribir los datos actualizados en Google Sheets
sheet.update(data)

print("Resultados de DOA guardados correctamente en Google Sheets.")
df_simulations.head()

[]


c:\Users\abell\Desktop\DSP\tp_dsp_tdoa\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\abell\Desktop\DSP\tp_dsp_tdoa\.venv\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


IndexError: list index out of range